In [1]:
import xlrd
import pandas as pd

In [2]:
def clean_data(sex='male'):
    '''
    Args：
        sex:either male or female
    Returns:
        print summary of data in console and export cleaned data in csv format
    '''
    
    if sex=='male':
        in_path_bmi='raw_data/Indicator_BMI male ASM.xlsx'
        in_path_bp='raw_data/Indicator_SBP male ASM.xlsx'
        out_path='male_combined.csv'
    elif sex=='female':
        in_path_bmi='raw_data/Indicator_BMI female ASM.xlsx'
        in_path_bp='raw_data/Indicator_SBP female ASM.xlsx'
        out_path='female_combined.csv'        
    else:
        raise KeyError('sex arg is invalid: female/male')
    
    
    def extract_reshape(sheet,val_title):
        '''
        Args:
            sheet: defined xlrd sheet
            val_title: column name of value
        Returns:
            reshaped pandas dataframe
        '''
        
        ### Read data from excel 
        df={}
        for col_idx in range(sheet.ncols):
            name=sheet.col_values(col_idx)[0]
            data=sheet.col_values(col_idx)[1:]
            if col_idx==0:
                df['Country']=pd.Series(data)
            else:
                df[int(name)]=pd.Series(data)
        df=pd.DataFrame(df)
        df=df.set_index('Country')

        ### Reshape data
        df=df.stack()
        df=df.reset_index()
        df.columns=['Country', 'Year', val_title]
        df=df.set_index(['Country', 'Year'])

        return df
        
    # Extract and clean bmi data
    book=xlrd.open_workbook(in_path_bmi)
    sheet=book.sheet_by_index(0)
    df_bmi=extract_reshape(sheet,'BMI')
    
    # Extract and clean bp data
    book=xlrd.open_workbook(in_path_bp)
    sheet=book.sheet_by_index(0)
    df_bp=extract_reshape(sheet,'BP')
    
    # Merge and map with continent
    continent=pd.read_csv('raw_data/Countries-Continents-csv.csv')[['Continent','Country']]
    continent=continent.set_index('Country')
    df=df_bmi.merge(df_bp,how='inner',left_index=True,right_index=True).merge(continent,how='left',left_index=True,right_index=True).fillna(0)
    df=df[df['Continent']!=0]
    df=df.reset_index()
    
    df.to_csv(out_path,index=False)
    
    print(sex,'data cleaned and exported')
    print()
    print(df.head())
    print('Min BMI: ',df['BMI'].min())
    print('Max BMI: ',df['BMI'].max())
    print('Min BP: ',df['BP'].min())
    print('Max BP: ',df['BP'].max())
    
    return df

In [3]:
df=clean_data('male')

male data cleaned and exported

       Country  Year       BMI        BP Continent
0  Afghanistan  1984  21.42734  125.3442      Asia
1  Afghanistan  1986  21.40132  125.4051      Asia
2  Afghanistan  1987  21.37679  125.4011      Asia
3  Afghanistan  1988  21.34018  125.3434      Asia
4  Afghanistan  1989  21.29845  125.2586      Asia
Min BMI:  19.01394
Max BMI:  33.89634
Min BP:  118.5221
Max BP:  143.1237


In [4]:
df=clean_data('female')

female data cleaned and exported

       Country  Year       BMI        BP Continent
0  Afghanistan  1984  20.60867  122.9095      Asia
1  Afghanistan  1986  20.68983  123.2892      Asia
2  Afghanistan  1987  20.70902  123.4189      Asia
3  Afghanistan  1988  20.71512  123.5137      Asia
4  Afghanistan  1989  20.71421  123.5832      Asia
Min BMI:  18.47166
Max BMI:  35.02294
Min BP:  110.3353
Max BP:  139.4559
